In [ ]:
from core.prepare import prepare_documents, build_index
from core.embedding import get_embed_model
from core.utils import get_keywords_from_llama
from core.query import init_chat_engine
from core.smart_chat import smart_chat_turn
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
import json, nest_asyncio

nest_asyncio.apply()

with open("sample.json", "r", encoding="utf-8") as f:
    recipe_data = json.load(f)

Settings.llm = Ollama(model="tinyllama:1.1b", request_timeout=600.0)
Settings.embed_model = get_embed_model("nomic-embed-text")
Settings.chunk_size = 256

documents = prepare_documents(recipe_data)
index = build_index(documents)

100%|██████████| 20/20 [00:12<00:00,  1.58it/s]


[INFO] Ingestion 完成，生成 20 个节点


In [2]:
# 模拟几轮问答
chat_engine = init_chat_engine(index)

print("=" * 20, "Smart Chat", "=" * 20)
queries = [
    "Can you recommend something with lemon and garlic?",
    "What dishes are similar to chicken curry?",
    "Can I make this for 10 people?",
    "How do I cook the drumsticks?"
]

for q in queries:
    print(f"\nUser: {q}")
    response = smart_chat_turn(q, chat_engine, index, embed_model=Settings.embed_model)
    print(f"Bot: {response}")

==================== Smart Chat ====================

User: Can you recommend something with lemon and garlic?
[INFO] Detected label: recommend
[DEBUG] Raw response: Extracting 3 keywords from this input: "Can you recommend something with lemon and garlic?" would yield the following output:

Keywords: lemon, garlic

Note that the returned keywords are not in any specific order and can be used for further processing.
Bot: Based on your input, here are some suggested recipes:
Sorry, I couldn't find any recipes matching your ingredients.

User: What dishes are similar to chicken curry?
[INFO] Detected label: recommend
[DEBUG] Raw response: [
    "chiken",
    "curry"
]
[WARN] No keywords found.
Bot: Based on your input, here are some suggested recipes:
Sorry, I couldn't find any recipes matching your ingredients.

User: Can I make this for 10 people?
[INFO] Detected label: recommend
[DEBUG] Raw response: Keywords: food-for-10people
Bot: Based on your input, here are some suggested recipes